<a href="https://colab.research.google.com/github/anthonyhu25/Variance-Reduction-Metropolis/blob/main/Variance_Reduction_for_Metropolis_Hastings_Example_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import random
from numpy import linalg
import math
import scipy
import scipy.stats
import matplotlib.pyplot as plt
from scipy.stats import rv_continuous, rv_discrete
from scipy.stats._distn_infrastructure import rv_frozen
from scipy.special import logsumexp
import scipy.integrate
import warnings
import sys
import statistics
import pandas as pd
from IPython.display import display, Math, HTML

This notebook, as well as the code in the other notebooks in this directory, will come from [this paper](https://arxiv.org/pdf/2203.02268).

# Example 3.1: Simulated Data Example: Gaussian Target


The target distribution of this example is a d-variate standard Gaussian distribution $N(0_{d}, I_{d})$ with a proposal distribution $q(y|x) ∼ N(y|x, c^{2}I_{d})$ where $c^{2} = 2.38^{2}/d$ for the Random Metropolis Walk case. We are interested in estimating the expected value of the first coordinate of the target, so $F(x) = x^{(1)}$



There are a couple of things I must note about the setup to this problem:
1. The coefficient-less estimator of $F: \mu_{n,G}(F):= \frac{1}{n}\sum_{i=0}^{n-1}[F(x_{i}) + \int α(x_{i}, y)(G(y) - G(x))q(y|x_{i})dy]$ needs a specified function $G(x)$ and also analytically evaluate the integral inside the estimator.

To first get an estimate $G$ to approximate $F$ (which can be estimated by expectation of $F$ with respect to target distribution $\pi(F)$), we need a Gaussian approximation of $\pi(x)$ first. We hope that $F_{\pi ̃}$ is a good approximation of the ideal function $F ^{ ̃}$, which is also an estimate of $F$. For this estimation, we set $G$

To estimate the integral $∫ \alpha(x_{i}, y)(G(y) - G(x))q(y|x_{i})dy$, we use Monte-Carlo estimates $α(x_{i}, y_{i})(G(y_{i}) - G(x_{i})), y_{i} \sim q(y|x_{i})$. To further reduce the variance of this estimator (since the Monte-Carlo estimates of the integral can have a high variance) we add in control variate $h(x_{i}, y)$ and $E_{q(y|x_{i})}(h(x_{i},y))$. Note that these terms $E_{q(y|x_{i})}(h(x_{i},y))$ and $h(x_{i},y)$ are static control variates, and also depends on the Gaussian approximation of $\pi(x)$.

So, to estimate the coefficient-less estimator of $F$ above, we use Monte-Carlo methods and use:

$\mu_{n, G}(F) := \frac{1}{n}\sum_{i=0}^{n-1}[F(x_{i}) + \alpha(x_{i}, y_{i})(G(y_{i}) - G(x_{i})) + h(x_{i}, y_{i}) - E_{q(y|x_{i})}[h(x_{i}, y)]]$

2. To obtain our static control variate $h(x_{i}, y)$, we first need Gaussian approximations of our target $\pi(x)$ and proposal $q(y|x)$ - let us name them $\pi^{̃}(x)$ and $q^{̃}(y|x)$ respectively - and the function $G(x)$. Then, we set $h(x,y)$ to be the product of the Metropolis-Hastings acceptance ratio between $\pi^{̃}(x)$ and $q^{̃}(y|x)$, and the difference between $G(y)$ and $G(x)$. Formally,

$h(x,y) = min(1, r^{̃}(x,y))[G(y)-G(x)]$

where $r^{̃}(x,y) = \frac{\pi^{̃}(y)q^{̃}(x|y)}{\pi^{̃}(x)q^{̃}(y|x)}$

We hope that the acceptance ratio of the Gaussian approximations also approximates the true acceptance ratio between the proposal and the density distributions.